# Generate Vertical Integrals

In [1]:
import glob
import os.path
import pprint

import distributed
import ncar_jobqueue
import xarray as xr
import yaml

from utils import path_replace

/glade/work/klindsay/miniconda3/envs/GCB_2022/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
xr.set_options(keep_attrs=True);

In [3]:
with open("GCB_metadata.yaml", mode="r") as fptr:
    GCB_metadata = yaml.safe_load(fptr)
pprint.pprint(GCB_metadata)

{'A': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001',
                 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.002']},
 'B': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRC.001']},
 'C': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRC.001']},
 'D': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRD.001',
                 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRD.002']}}


In [4]:
tseries_root = "/glade/campaign/cesm/development/bgcwg/projects/GCB_2022"

In [5]:
def gen_zint_driver(GCB_name, CESM_cases):
    print(GCB_name)
    print(CESM_cases)

    for varname in ["DIC", "spC", "diatC", "diazC", "zooC"]:
        for case in CESM_cases:
            dir = f"{tseries_root}/{case}/output/ocn/proc/tseries/month_1"
            case_paths = glob.glob(f"{dir}/{case}.pop.h.{varname}.*.nc")
            case_paths.sort()
            for path in case_paths:
                gen_zint(path, varname)


def gen_zint(path_in, varname):
    ds = xr.open_dataset(path_in, chunks={"time": 12})
    da = ds[varname]

    reduce_dim = da.dims[1]
    if reduce_dim == "z_t":
        weight = ds["dz"]
    elif reduce_dim == "z_t_150m":
        weight = xr.DataArray(
            ds["dz"][0:15].values, coords=[ds["z_t_150m"]], attrs=ds["dz"].attrs
        )
    else:
        raise ValueError(f"unknown reduce_dim={reduce_dim}")

    da_out = (da * weight).sum(dim=reduce_dim)

    ssh_path = path_replace(path_in, base_replace={varname: "SSH"})
    ds_ssh = xr.open_dataset(ssh_path, chunks={"time": 12})
    da_out += da[:,0,:] * ds_ssh["SSH"]

    da_out.attrs = da.attrs
    da_out.attrs["long_name"] = da.attrs["long_name"] + " Vertical Integral"
    da_out.attrs["units"] = ds["dz"].attrs["units"] + " " + da.attrs["units"]
    da_out.attrs["grid_loc"] = "2110"
    da_out.encoding = da.encoding
    da_out.encoding["coordinates"] = "TLONG TLAT time"

    ds_out = ds.drop_vars(varname)
    ds_out[varname+"_zint"] = da_out

    # ensure NaN _FillValues do not get generated
    for d in [ds_out.variables, ds_out.coords]:
        for var in d:
            if "_FillValue" not in ds_out[var].encoding:
                ds_out[var].encoding["_FillValue"] = None

    path_out = path_replace(path_in, base_replace={varname: varname + "_zint"})

    print(f"writing {os.path.basename(path_out)}")
    ds_out.to_netcdf(path_out)

# Obtain Computational Resources

In [6]:
cluster = ncar_jobqueue.NCARCluster(
    cores=1,  # The number of cores you want
    memory='4GB',  # Amount of memory
    processes=1,  # How many processes
    walltime='01:00:00',  # Amount of wall time
)

cluster.scale(8)

client = distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.39:41138,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
for GCB_name in GCB_metadata:
    gen_zint_driver(GCB_name, GCB_metadata[GCB_name]["cases"])

A
['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001', 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.002']
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.177801-180212.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.180301-182712.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.182801-185212.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.185301-187712.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.187801-190212.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.190301-192712.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.192801-195212.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint.195301-197712.nc
writing g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.DIC_zint

# Release Computational Resources

In [8]:
client.close()
cluster.close()
